# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [1]:
###
# Add code here
import open3d as o3d
import numpy as np
import copy
from scipy.spatial import KDTree
import random

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

In [3]:
####
# Downsample and find features here
####

source.voxel_down_sample(voxel_size)
target.voxel_down_sample(voxel_size)

#draw_registrations(source, target)


source.estimate_normals()
target.estimate_normals()


print(type(source))
feature_source = o3d.pipelines.registration.compute_fpfh_feature(source, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))
feature_target = o3d.pipelines.registration.compute_fpfh_feature(target, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))

<class 'open3d.cpu.pybind.geometry.PointCloud'>


In [4]:
####
# Call RANSAC here
corr_length = 0.9
distance_threshold = voxel_size * 1.5
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
#point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

In [5]:
ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source, target, 
    feature_source, feature_target, 
    distance_threshold,
    1,
    point_to_point,
    checkers = checker_list)

print(ransac_result.transformation)

####

[[-0.49438555  0.86573007 -0.07806645  0.80076054]
 [-0.20155376 -0.02680927  0.97911049  0.45349305]
 [ 0.84555249  0.49979266  0.18774526 -1.28964848]
 [ 0.          0.          0.          1.        ]]


In [13]:
def global_registration(source, target, source_feature, target_feature, ransac_matrix, max_iter=100000):
    tree = KDTree(np.transpose(target_feature.data))
    distances, matches = tree.query(np.transpose(source_feature.data), 1)
    print("Tree created")
    
    source_points = np.asarray(source.points)
    target_points = np.asarray(target.points)
    source_center = source.get_center()
    target_center = target.get_center()
    
    centered_source = source_points-source_center
    centered_target = target_points-target_center
    
    similarity = []
    results = []
    for i in range(max_iter):
        rand_indeces = [random.randint(0, len(centered_source)) for i in range(3)]
        p_prime = np.array([centered_source[i] for i in rand_indeces])
        q_prime = np.array([centered_target[matches[i]] for i in rand_indeces])

        cov_matrix = np.matmul(p_prime, np.transpose(q_prime))

        u,w,v = np.linalg.svd(cov_matrix)

        R = np.matmul(u, np.transpose(v))

        T = target_center - np.matmul(R, source_center)

        transf = np.vstack((np.hstack((R, np.transpose(np.array([T])))), np.array([[0, 0, 0, 1]])))
        
        #source_temp.transform(transf)
        
        #compare transf with ransac matrix
        similarity.append(np.sum(np.absolute(target_points-np.asarray(source.transform(transf)))))
        if i%5000 == 0:
            print(i, similarity[-1])
        results.append(transf)
        
    print(min(similarity))
    return results[np.argmin(similarity)]
 

our_matrix = global_registration(source, target, feature_source, feature_target, ransac_result.transformation)

Tree created


TypeError: unsupported operand type(s) for -: 'float' and 'open3d.cpu.pybind.geometry.PointCloud'

In [10]:
# ransac_result = ...
draw_registrations(source, target, our_matrix, True)